<div style="display: flex; background-color: #ad283c;" >
<h1 style="margin: auto; padding: 30px; ">ANALYSE DU STOCK ET DES VENTES DU SITE BOTTLENECK</h1>
</div>

# OBJECTIF DE CE NOTEBOOK

Les objectifs sont les suivants :

- Explorer les données fournies
- Améliorer le workflow
- Repérer les erreurs dans la base de données



<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 1 - Importation des librairies et chargement des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.1 - Importation des librairies</h3>
</div>

In [1]:
#Importation de la librairie Pandas
import pandas as pd

# Importation de numpy
import numpy as np

In [2]:
#Importation de la librairie plotly express


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.2 - Chargements des fichiers</h3>
</div>

In [2]:
# Desactivation des warnings pour l'import des fichiers
import warnings
from openpyxl.utils.exceptions import InvalidFileException

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [3]:
#Importation du fichier web.xlsx
df_web = pd.read_excel("./source_data/web.xlsx", engine="openpyxl")
#Importation du fichier erp.xlsx
df_erp = pd.read_excel("./source_data/erp.xlsx", engine="openpyxl")
#Importation du fichier liaison.xlsx
df_liaison = pd.read_excel("./source_data/liaison.xlsx", engine="openpyxl")

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 2 - Analyse exploratoire des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1 - Analyse exploratoire du fichier erp.xlsx</h3>
</div>

In [4]:
#Afficher les dimensions du dataset
print(f"Le tableau comporte {df_erp.shape[0]} observation(s) ou article(s)")
print(f"Le tableau comporte {df_erp.shape[1]} colonne(s)")

Le tableau comporte 825 observation(s) ou article(s)
Le tableau comporte 6 colonne(s)


In [5]:
#Consulter les colonnes
#La nature des données dans chacune des colonnes
#Le nombre de valeurs présentes dans chacune des colonnes
print(10 * "-" + " Informations du fichier ERP " + 10 * "-" + "\n")
df_erp.info()


---------- Informations du fichier ERP ----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      825 non-null    int64  
 1   onsale_web      825 non-null    int64  
 2   price           825 non-null    float64
 3   stock_quantity  825 non-null    int64  
 4   stock_status    825 non-null    object 
 5   purchase_price  825 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 38.8+ KB


In [6]:
#Afficher les 5 premières lignes de la table
df_erp.head()

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price
0,3847,1,24.2,16,instock,12.88
1,3849,1,34.3,10,instock,17.54
2,3850,1,20.8,0,outofstock,10.64
3,4032,1,14.1,26,instock,6.92
4,4039,1,46.0,3,outofstock,23.77


In [7]:
#Vérifier si il y a des lignes en doublon dans la colonne product_id
df_erp[df_erp.duplicated(subset=['product_id'], keep=False)]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price


In [8]:
#Afficher les valeurs distinctes de la colonne stock_status
print(pd.unique(df_erp['stock_status']))

#À quelle(s) autre(s) colonne(s) sont-elles liées ?
# stock_quantity

['instock' 'outofstock']


In [9]:
#Création d'une colonne "stock_status_2"
#La valeur de cette deuxième colonne sera fonction de la valeur dans la colonne "stock_quantity"
#Si la valeur de la colonne "stock_quantity" est nulle, renseigner "outofstock" sinon mettre "instock"
df_erp['stock_status_2'] = np.where(
    df_erp['stock_quantity'] > 0,
    'instock',
    'outofstock'
)
df_erp['stock_status_2']

0         instock
1         instock
2      outofstock
3         instock
4         instock
          ...    
820       instock
821       instock
822       instock
823       instock
824       instock
Name: stock_status_2, Length: 825, dtype: object

In [10]:
#Vérifions que les 2 colonnes sont identiques:
#Les 2 colonnes sont strictement identiques si les valeurs de chaque ligne sont strictement identiques 2 à 2
#La comparaison de 2 colonnes peut se réaliser simplement avec l'instruction ci-dessous:
df_erp["stock_status"] == df_erp["stock_status_2"]

#Le résultat est l'affichage de True ou False pour chacune des lignes du dataset
#C'est un bon début, mais difficile à exploiter

0       True
1       True
2       True
3       True
4      False
       ...  
820     True
821     True
822     True
823     True
824     True
Length: 825, dtype: bool

In [11]:
#Mais il est possible de synthétiser ce résultat en effectuant la somme de cette colonne:
#True vaut 1 et False 0
#Nous devrions obtenir la somme de 824 qui correspond au nombre de lignes dans ce dataset
print(f"{(df_erp["stock_status"] == df_erp["stock_status_2"]).sum()} valeurs correctes dans la colonne stock_status sur {len(df_erp)}")

823 valeurs correctes dans la colonne stock_status sur 825


In [12]:
#Si les colonnes ne sont absolument pas identiques ligne à ligne alors identifier la ligne en écart
incoherence_stock: pd.DataFrame = df_erp[df_erp['stock_status'] != df_erp['stock_status_2']]
incoherence_stock

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price,stock_status_2
4,4039,1,46.0,3,outofstock,23.77,instock
398,4885,1,18.7,0,instock,9.66,outofstock


In [13]:
#Corriger la ou les données incohérentes
df_erp.rename(columns={'stock_status': 'stock_status_original'}, inplace=True)
df_erp.rename(columns={'stock_status_2': 'stock_status'}, inplace=True)
df_erp
# Je ne modifie pas les colonnes d'origine pour garder une cohérence des données
# A la place je renomme les colonnes pour pouvoir les manipuler avec plus de facilités après

,product_id,onsale_web,price,stock_quantity,stock_status_original,purchase_price,stock_status
0,3847,1,24.2,16,instock,12.88,instock
1,3849,1,34.3,10,instock,17.54,instock
2,3850,1,20.8,0,outofstock,10.64,outofstock
3,4032,1,14.1,26,instock,6.92,instock
4,4039,1,46.0,3,outofstock,23.77,instock
...,...,...,...,...,...,...,...
820,7203,0,45.0,30,instock,23.48,instock
821,7204,0,45.0,9,instock,24.18,instock
822,7247,1,54.8,6,instock,27.18,instock
823,7329,0,26.5,14,instock,13.42,instock


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1 - Analyse exploratoire de chaque variable du fichier erp.xlsx</h3>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.1 - Analyse de la variable PRIX</h3>
</div>

In [14]:
###############
## LES PRIX  ##
###############

#Vérification des prix: Y a t-il des prix non renseignés, négatifs ou nuls?
#Afficher le ou les prix non renseignés dans la colonne "price"
prix_invalides: pd.Series = df_erp['price'] <= 0
prix_manquant: pd.Series = df_erp['price'].isna()
filtre_prix_incorrect = (prix_invalides | prix_manquant)
nombre_articles_invalides: np.int64 = filtre_prix_incorrect.sum()
print(f"Nombres d'articles avec un prix mal renseigné: {nombre_articles_invalides}") 

#Afficher le prix minimum de la colonne "price"
print(f"Prix minimum dans la colonne price : {df_erp['price'].min()}")

#Afficher le prix maximum de la colonne "price"
print(f"Prix maximum dans la colonne price : {df_erp['price'].max()}")

#Afficher les prix inférieurs à 0 (qu'est-ce qu'il faut en faire ?)
df_erp[filtre_prix_incorrect]


Nombres d'articles avec un prix mal renseigné: 3
Prix minimum dans la colonne price : -20.0
Prix maximum dans la colonne price : 225.0


,product_id,onsale_web,price,stock_quantity,stock_status_original,purchase_price,stock_status
151,4233,0,-20.0,0,outofstock,10.33,outofstock
469,5017,0,-8.0,0,outofstock,4.34,outofstock
739,6594,0,-9.1,19,instock,4.61,instock


Plusieurs solutions s'offrent à nous pour gérer ces cas :
- Les retirer de la vente termporairement 
- Supprimer les lignes et retirer de la vente définitevement (solution la moins envisageable)
- Mettre des prix justes au dessus du prix d'achat afin de pouvoir vendre ces produits

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.2 - Analyse de la variable STOCK</h3>
</div>

In [15]:
#######################
### stock_quantity  ###
#######################

#Vérification de la colonne stock quantity
#Afficher la quantité minimum de la colonne "stock_quantity"
print(f"Nombre minimum de stock : {df_erp['stock_quantity'].min()}")

#Afficher la quantité maximum de la colonne "stock_quantity"
print(f"Nombre maximum de stock : {df_erp['stock_quantity'].max()}")

#Afficher les stocks inférieurs à 0 (qu'est-ce qu'il faut en faire ?)
print("\nStock avec des valeurs absurdes :")
df_erp[df_erp['stock_quantity'] < 0]


Nombre minimum de stock : -10
Nombre maximum de stock : 145

Stock avec des valeurs absurdes :


,product_id,onsale_web,price,stock_quantity,stock_status_original,purchase_price,stock_status
449,4973,0,10.0,-10,outofstock,4.96,outofstock
573,5700,1,44.5,-1,outofstock,22.30,outofstock


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.3 - Analyse de la variable ONSALE_WEB</h3>
</div>

In [16]:
# Vérification de la colonne onsale_web et des valeurs qu'elle contient. Que signifient-elles?
print(pd.unique(df_erp["onsale_web"]))

[1 0]


Ne contenant que des 0 ou des 1, cette colonne peut signifier plusieurs choses :
- De savoir si le produit est en vente actuellement ou non
- De savoir si le produit est en solde

In [ ]:
#Quelles sont les colonnes à conserver selon vous?

Les colonnes à conserver sont : 
- product_id
- onsale_web
- price
- stock_quantity
- purchase_price

stock_status peut être théoriquement supprimé car c'est une valeur déduite de la colonne stock_quantity

In [17]:
#Supprimer la colonne comportant le libellé "stock_status_2" car elle est redondante 
#avec la colonne "stock_status".
df_erp = df_erp.drop(columns=['stock_status_original'])
df_erp.head()

,product_id,onsale_web,price,stock_quantity,purchase_price,stock_status
0,3847,1,24.2,16,12.88,instock
1,3849,1,34.3,10,17.54,instock
2,3850,1,20.8,0,10.64,outofstock
3,4032,1,14.1,26,6.92,instock
4,4039,1,46.0,3,23.77,instock


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.4 - Analyse de la variable prix d'achat</h3>
</div>

In [22]:
######################
##   prix d'achat   ##
######################

#Vérification de la colonne purchase_price : 
#Afficher le ou les prix non renseignés dans la colonne "purchase_price"
print("Liste des lignes avec des prix non rensiegnés :")
print(df_erp[df_erp['purchase_price'].isna()])

#Afficher le prix minimum de la colonne "purchase_price"
print(f"\nPrix minimum dans la colonne purchase_price : {df_erp['purchase_price'].min()}")

#Afficher le prix maximum de la colonne "purchase_price"
print(f"Prix maximum dans la colonne purchase_price : {df_erp['purchase_price'].max()}")


Liste des lignes avec des prix non rensiegnés :
Empty DataFrame
Columns: [product_id, onsale_web, price, stock_quantity, purchase_price, stock_status]
Index: []

Prix minimum dans la colonne purchase_price : 2.74
Prix maximum dans la colonne purchase_price : 137.81


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.2 - Analyse exploratoire du fichier web.xlsx</h3>
</div>
 

In [26]:
#Dimension du dataset
#Nombre d'observations
print(f"Nombre de lignes / observations présentes dans le fichier web : {df_web.shape[0]}")

#Nombre de caractéristiques
print(f"Nombre de colonnes / caractéristiques présentes dans le fichier web : {df_web.shape[1]}")


Nombre de lignes / observations présentes dans le fichier web : 1513
Nombre de colonnes / caractéristiques présentes dans le fichier web : 29


In [27]:
#Consulter le nombre de colonnes
#La nature des données dans chacune des colonnes
#Le nombre de valeurs présentes dans chacune des colonnes
df_web.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   sku                    1428 non-null   object        
 1   virtual                1513 non-null   int64         
 2   downloadable           1513 non-null   int64         
 3   rating_count           1513 non-null   int64         
 4   average_rating         1430 non-null   float64       
 5   total_sales            1430 non-null   float64       
 6   tax_status             716 non-null    object        
 7   tax_class              0 non-null      float64       
 8   post_author            1430 non-null   float64       
 9   post_date              1430 non-null   datetime64[ns]
 10  post_date_gmt          1430 non-null   datetime64[ns]
 11  post_content           0 non-null      float64       
 12  product_type           1429 non-null   object        
 13  pos

In [43]:
# Analyse supplémentaire avant de décider quelles colonnes conservées
# Vérification du contenu de certains attributs
print(f"Valeurs uniques de virtual : {pd.unique(df_web['virtual'])}")
print(f"Valeurs uniques de downloadable : {pd.unique(df_web['downloadable'])}")
print(f"Valeurs uniques de rating_count : {pd.unique(df_web['rating_count'])}")
print(f"Valeurs uniques de average_rating : {pd.unique(df_web['average_rating'])}")
print(f"Valeurs uniques de tax_status : {pd.unique(df_web['tax_status'])}")
print(f"Valeurs uniques de post_author : {pd.unique(df_web['post_author'])}")
print(f"Valeurs uniques de post_content : {pd.unique(df_web['post_content'])}")
print(f"Valeurs uniques de post_status : {pd.unique(df_web['post_status'])}")
print(f"Valeurs uniques de comment_status : {pd.unique(df_web['comment_status'])}")
print(f"Valeurs uniques de ping_status : {pd.unique(df_web['ping_status'])}")
print(f"Valeurs uniques de post_parent : {pd.unique(df_web['post_parent'])}")
print(f"Valeurs uniques de menu_order : {pd.unique(df_web['menu_order'])}")
print(f"Valeurs uniques de post_type : {pd.unique(df_web['post_type'])}")
print(f"Valeurs uniques de comment_count : {pd.unique(df_web['comment_count'])}")

Valeurs uniques de virtual : [0]
Valeurs uniques de downloadable : [0]
Valeurs uniques de rating_count : [0]
Valeurs uniques de average_rating : [ 0. nan]
Valeurs uniques de tax_status : [nan 'taxable']
Valeurs uniques de post_author : [ 2. nan  1.]
Valeurs uniques de post_content : [nan]
Valeurs uniques de post_status : ['publish' nan]
Valeurs uniques de comment_status : ['closed' nan]
Valeurs uniques de ping_status : ['closed' nan]
Valeurs uniques de post_parent : [ 0. nan]
Valeurs uniques de menu_order : [ 0. nan]
Valeurs uniques de post_type : ['attachment' 'product' nan]
Valeurs uniques de comment_count : [ 0. nan]


In [42]:
# Vérification des lignes avec des sku vides
df_web[df_web['sku'].isna()]

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
8,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1429,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1432,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Selon vous, quelles sont les colonnes à conserver ?
Après analyse, voici les colonnes que je vais conserver : 
- sku (Identifiant du produit et fait le lien avec l'ERP)
- total_sales (Représentant les ventes totales du produit)
- post_date_gmt (Début de vente du produit à GMT+0)
- product_type (Permet de différencier le type de produit pour les catégoriser)
- post_title (Nom du produit, permettra de trouver facilement les noms des produits qui vont sortir suite à l'analyse)
- post_type (Afin de ne récupérer que les lignes concernant les produits, sera vite abandonnée)

In [49]:
#Si vous avez défini des colonnes à supprimer, effectuer l'opération
df_web_reduce = df_web[['sku', 'total_sales', 'post_date_gmt', 'product_type', 'post_title', 'post_type']]
df_web_clean = df_web_reduce[df_web_reduce['post_type'] == 'product'].drop(columns=['post_type'])
df_web_clean.head()

,sku,total_sales,post_date_gmt,product_type,post_title
2,14692,5.0,2019-03-19 09:06:47,Vin,Château Fonréaud Bordeaux Blanc Le Cygne 2016
4,15328,2.0,2019-03-27 17:05:09,Vin,Agnès Levet Côte Rôtie Maestria 2017
6,16515,10.0,2018-06-02 07:31:31,Vin,Château Turcaud Bordeaux Rouge Cuvée Majeure 2018
11,16585,15.0,2018-02-16 13:03:16,Vin,Xavier Frissant Touraine Sauvignon 2019
14,12869,7.0,2019-03-28 13:29:35,Vin,Stéphane Tissot Arbois D.D. 2016


In [50]:
#Visualisation des valeurs de la colonne sku
#Quelles sont les valeurs qui ne semblent pas respecter la régle de codification?
print(f"Valeurs uniques de sku : {pd.unique(df_web_clean['sku'])}")

Valeurs uniques de sku : [14692 15328 16515 16585 12869 15575 14338 16560 15361 15022 16342 16029
 13754 14680 9636 13849 15481 15448 15441 804 16071 12882 16053 13766
 12640 15476 16038 14864 16044 15324 15413 13809 15895 15849 12315 15934
 15148 15781 15106 15490 14507 16307 13736 16037 12587 16305 16131 13435
 15758 14509 14768 16505 15871 11602 13127 13520 13032 15436 15910 16263
 15138 16580 13905 13557 14975 15341 15415 16065 15479 16151 15127 15140
 15779 9937 15281 15315 15668 15161 15792 15921 15870 15690 15561 15539
 16320 3509 2534 11586 14819 15705 15254 15554 14451 16274 14696 14573
 16238 15834 15141 15038 15238 16586 15664 14527 15707 15269 14845 14700
 15440 15759 11277 15676 15731 16525 15785 14699 15718 14372 15881 15426
 16023 14661 15675 15351 12791 13958 '13127-1' 11849 15461 15577 12639
 15466 15184 13904 15793 15612 15425 14599 15033 16449 15829 13965 16306
 15303 13996 14679 15318 13599 15206 16153 16067 15241 16264 14915 16289
 15229 15713 15654 15879 12586 157

In [51]:
#Si vous avez identifié des codes articles ne respectant pas la régle de codification, consultez-les
df_web_clean['sku_numeric'] = pd.to_numeric(df_web_clean['sku'], errors='coerce')
df_web_clean[df_web_clean['sku_numeric'].isna()]

,sku,total_sales,post_date_gmt,product_type,post_title,sku_numeric
272,13127-1,4.0,2020-06-09 13:42:04,Vin,Clos du Mont-Olivet Châteauneuf-du-Pape 2007,NaN
1084,NaN,-56.0,2018-08-08 09:23:43,Vin,Pierre Jean Villa Condrieu Jardin Suspendu 2018,NaN
1087,NaN,-17.0,2018-07-31 10:07:23,Vin,Pierre Jean Villa Côte Rôtie Fongeant 2017,NaN
1387,bon-cadeau-25-euros,7.0,2018-06-01 11:53:46,NaN,Bon cadeau de 25€,NaN


In [62]:
#Identifier les lignes sans code article
df_web_clean[df_web_clean['sku'].isna()]

,sku,total_sales,post_date_gmt,product_type,post_title,sku_numeric
1084,NaN,-56.0,2018-08-08 09:23:43,Vin,Pierre Jean Villa Condrieu Jardin Suspendu 2018,NaN
1087,NaN,-17.0,2018-07-31 10:07:23,Vin,Pierre Jean Villa Côte Rôtie Fongeant 2017,NaN


In [60]:
#Pour les codes articles identifiés, réaliser une analyse et définir l'action à entreprendre

- Pour le sku 13127-1, il faut lui fournir un sku valide
- Pour les bons d'achats, fournir un sku et créer une catégorie, car la vente de bon peut être de nouveau possible
- Pour les NaN, il faut vérifier si c'est un retour totale d'une commande, une erreur énorme de saisie ? Pour les analyses il faudrait les isoler, voir les supprimer car impossible de retrouver leur code et donc leur prix

In [66]:
#La clé pour chaque ligne est-elle unique? autrement dit, y a-t-il des doublons?
df_web_clean[df_web_clean['sku'].duplicated(keep=False)]

,sku,total_sales,post_date_gmt,product_type,post_title,sku_numeric
1084,NaN,-56.0,2018-08-08 09:23:43,Vin,Pierre Jean Villa Condrieu Jardin Suspendu 2018,NaN
1087,NaN,-17.0,2018-07-31 10:07:23,Vin,Pierre Jean Villa Côte Rôtie Fongeant 2017,NaN


In [75]:
#Les lignes sans code article semblent être toutes non renseignées
#Pour s'en assurer, réaliser les étapes suivantes:
#1 - Créer un dataframe avec uniquement les lignes sans code article
df_web_empty = df_web_reduce[df_web_reduce['sku'].isnull()]
#2 - Utiliser la fonction df.info() sur ce nouveau dataframe pour observer le nombre de valeurs renseignées dans chacune des colonnes
df_web_empty.info()
#3 - Que constatez-vous?
df_web_empty[~df_web_empty['total_sales'].isnull()]

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 8 to 1457
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   sku            0 non-null      object        
 1   total_sales    2 non-null      float64       
 2   post_date_gmt  2 non-null      datetime64[ns]
 3   product_type   2 non-null      object        
 4   post_title     2 non-null      object        
 5   post_type      2 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 4.6+ KB


,sku,total_sales,post_date_gmt,product_type,post_title,post_type
1084,NaN,-56.0,2018-08-08 09:23:43,Vin,Pierre Jean Villa Condrieu Jardin Suspendu 2018,product
1087,NaN,-17.0,2018-07-31 10:07:23,Vin,Pierre Jean Villa Côte Rôtie Fongeant 2017,product


Les articles sans sku (hormis les deux coquilles avec des ventes négatives) n'ont que des valeurs nulles. 
Ils peuvent donc être retirés car aucune informations n'est utile.

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.3 - Analyse exploratoire du fichier liaison.xlsx</h3>
</div>

In [76]:
#Dimension du dataset
#Nombre d'observations
print(f"Nombre de lignes / observations présentes dans le fichier liaison : {df_liaison.shape[0]}")

#Nombre de caractéristiques
print(f"Nombre de colonnes / caractéristiques présentes dans le fichier liaison : {df_liaison.shape[1]}")

Nombre de lignes / observations présentes dans le fichier liaison : 825
Nombre de colonnes / caractéristiques présentes dans le fichier liaison : 2


In [77]:
#Consulter le nombre de colonnes
#La nature des données dans chacune des colonnes
#Le nombre de valeurs présentes dans chacune des colonnes
df_liaison.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_web      734 non-null    object
 1   product_id  825 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.0+ KB


In [102]:
#Les valeurs de la colonne "product_id" sont-elles toutes uniques?
print(f"{df_liaison['product_id'].duplicated().sum()} duplicata(s) dans la colonne 'product_id'.")
print(f"{df_liaison[df_liaison['product_id'].isna()]['product_id'].sum()} valeur(s) NaN dans la colonne 'product_id'.")

0 duplicata(s) dans la colonne 'product_id'.
0 valeur(s) NaN dans la colonne 'product_id'.


Oui les product_id sont bien uniques, car la somme de la recherche de duplicata est bien nulle

In [99]:
#Les valeurs de la colonne "id_web" sont-elles toutes uniques?
print(f"{df_liaison['id_web'].duplicated().sum()} duplicata(s) dans la colonne 'id_web'.")
df_liaison[df_liaison['id_web'].duplicated()]

90 duplicata(s) dans la colonne 'id_web'.


,id_web,product_id
49,NaN,4090
50,NaN,4092
119,NaN,4195
131,NaN,4209
151,NaN,4233
...,...,...
817,NaN,7196
818,NaN,7200
819,NaN,7201
820,NaN,7203


In [100]:
print(f"{df_liaison[~df_liaison['id_web'].isna()]['id_web'].duplicated().sum()} duplicata(s) dans la colonne 'id_web' sans NaN.")

0 duplicata(s) dans la colonne 'id_web' sans NaN.


La colonne `id_web` n'est pas aussi propre que sa voisine, il y a 90 lignes qui sont des duplicatas, mais avec un peu de recherche ce n'est que des valeurs NaN.
Si on omet ces valeurs, on a bien des valeurs uniques. 

In [ ]:
#Avons-nous des articles sans correspondance?

Comme vu précedemment, oui il y a des articles qui ont un id_web nul, donc impossible de faire la correspondance avec la base ERP.

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 3 - Jonction des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 3.1 - Jonction du fichier df_erp et df_liaison</h3>
</div>

In [103]:
#Fusion des fichiers df_erp et df_liaison
df_merge = pd.merge(df_erp, df_liaison, how="left", on="product_id")

In [105]:
#Y a t-il des lignes ne "matchant" pas entre les 2 fichiers?
df_merge.info()
df_merge

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      825 non-null    int64  
 1   onsale_web      825 non-null    int64  
 2   price           825 non-null    float64
 3   stock_quantity  825 non-null    int64  
 4   purchase_price  825 non-null    float64
 5   stock_status    825 non-null    object 
 6   id_web          734 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 45.2+ KB


,product_id,onsale_web,price,stock_quantity,purchase_price,stock_status,id_web
0,3847,1,24.2,16,12.88,instock,15298
1,3849,1,34.3,10,17.54,instock,15296
2,3850,1,20.8,0,10.64,outofstock,15300
3,4032,1,14.1,26,6.92,instock,19814
4,4039,1,46.0,3,23.77,instock,19815
...,...,...,...,...,...,...,...
820,7203,0,45.0,30,23.48,instock,NaN
821,7204,0,45.0,9,24.18,instock,NaN
822,7247,1,54.8,6,27.18,instock,13127-1
823,7329,0,26.5,14,13.42,instock,14680-1


Toutes les lignes matchent, car on obtient le même nombre de lignes.

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 3.2 - Jonction du fichier df_merge et df_web</h3>
</div>

In [113]:
#Fusionner les datasets df_merge et df_web
df_merge = df_merge.rename(columns={'id_web': 'sku'})
df_fusion = pd.merge(df_web_clean[~df_web_clean['sku'].isna()], df_merge, how="left", on="sku")
df_fusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   sku             714 non-null    object        
 1   total_sales     714 non-null    float64       
 2   post_date_gmt   714 non-null    datetime64[ns]
 3   product_type    713 non-null    object        
 4   post_title      714 non-null    object        
 5   sku_numeric     712 non-null    float64       
 6   product_id      714 non-null    int64         
 7   onsale_web      714 non-null    int64         
 8   price           714 non-null    float64       
 9   stock_quantity  714 non-null    int64         
 10  purchase_price  714 non-null    float64       
 11  stock_status    714 non-null    object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 67.1+ KB


In [ ]:
#Avons-nous des lignes sans correspondance?

Oui il y a des lignes sans correspondance, mais c'était à prévoir car la colonne id_web de df_liaison comportait des NaN, et que je n'ai pas fait de jointure sur ces valeurs ne sachant si elles correspondent.

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 4 - Analyse univariée des prix</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.1 - Exploration par la visualisation de données</h3>
</div>

In [40]:
#Création d'une boîte à moustache de la répartition des prix grâce à Pandas


In [41]:
#Autre méthode avec plotly express


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.2 - Exploration par l'utilisation de méthodes statistiques</h3>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.2.1 - Identification par le Z-index</h3>
</div>

In [42]:
#Calculer la moyenne du prix

#Calculer l'écart-type du prix

#Calculer le Z-score


In [43]:
#Quel est le seuil prix dont le z-score est supérieur à 3?


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.2.2 - Identification par l'intervalle interquartile</h3>
</div>

In [44]:
#Utilisation de la fonction "describe" de Pandas pour l'étude des mesures de dispersion


In [45]:
#Définir un seuil pour les articles "outliers" en prix


In [46]:
#Définir le nombre d'articles et la proportion de l'ensemble du catalogue "outliers"


In [47]:
#Selon vous, ces outliers sont-ils justifiés ? Comment le démontrer si cela est possible ?


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 5 - Analyse univariée du CA, des quantités vendues, des stocks et de la marge ainsi qu'une analyse multivariée  </h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.1 - Analyse des ventes en CA</h3>
</div>

In [48]:
##############################
# Calculer le CA du site web #
##############################

#Créer une colonne calculant le CA par article

#Calculer la somme de la colonne "ca_par_article"
#Ce résultat correspond au chiffre d'affaire du site web


In [49]:
###############################
# Palmarès des articles en CA #
###############################

#Effectuer le tri dans l'ordre décroissant du CA du dataset df_merge

#Réinitialiser l'index du dataset par un reset_index

#Afficher les 20 premiers articles en CA

#Graphique en barre des 20 premiers articles avec plotly express


In [50]:
#############################
# Calculer le 20 / 80 en CA #
#############################

#Créer une colonne calculant la part du CA de la ligne dans le dataset

#Créer une colonne réalisant la somme cumulative de la colonne précedemment créée

#Grâce aux deux colonnes créées précedemment, calculer le nombre d'articles représentant 80% du CA

#Afficher la proportion que représente ce groupe d'articles dans le catalogue entier du site web


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.2 - Analyse des ventes en quantité</h3>
</div>

In [51]:
#####################################
# Palmarès des articles en quantité #
#####################################

#Effectuer le tri dans l'ordre décroissant de quantités vendues du dataset df_merge

#Réinitialiser l'index du dataset par un reset_index

#Afficher les 20 premiers articles en quantité

#Graphique en barre des 20 premiers articles avec plotly express


In [52]:
#############################
# Calculer le 20 / 80 en CA #
#############################

#Créer une colonne calculant la part en quantité de la ligne dans le dataset

#Créer une colonne réalisant la somme cumulative de la colonne précedemment créée

#Grâce aux deux colonnes créées précedemment, calculer le nombre d'articles représentant 80% des ventes en quantité

#Afficher la proportion que représente ce groupe d'articles dans le catalogue entier du site web


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.3 - Analyse des stocks</h3>
</div>

In [53]:
######################################
# Calculer le nombre de mois de stock #
######################################

#Import de numpy 

#Création de la colonne Rotation de stock

#Remplacement des "inf" par 0

#Effectuer le tri dans l'ordre décroissant du nombre de mois de stock dans le dataset df_merge

#Graphique en barre du flop 20 des produits qui ont le plus de mois de stock

In [54]:
####################################
# Valorisation des stocks en euros #
####################################

#Création de la colonne Valorisation des stocks en euros

#Calculer la somme de la colonne "Valorisation_stock_euros"

In [55]:
##############################################
# Valorisation du nombre de produits en stock #
##############################################

#Calculer la somme de la colonne stock quantity

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.4 - Analyse du taux de marge</h3>
</div>

In [56]:
############################
# Analyse du taux de marge #
############################

#Création de la colonne Prix HT

#Création de la colonne Taux de marge

#Afficher le prix minimum de la colonne "taux_marge"

#Afficher le prix maximum de la colonne "taux_marge"


In [57]:
#Affichage de la ligne avec un taux de marge inférieur à 0


In [58]:
#Création d'un dataframe avec les taux positifs

#Afficher le prix minimum de la colonne "taux_marge"

#Afficher le prix maximum de la colonne "taux_marge"


In [59]:
#Création d'un dataframe avec le taux de marge moyen par type de produit

#Affichage dans un graphique du taux de marge par type de produit


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.5 - Analyse des corrélations entre les variables stock, sales et price</h3>
</div>

In [60]:
############################
# Analyse des corrélations #
############################

#Importation de Seaborn

#Création d'une heatmap de corrélation avec les variables stock, sales et price
#On peut également créer un mask pour n'afficher qu'une demi heatmap

In [61]:
#Que peut-on conclure des corrélations ?

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.6 - Mise à disposition de la nouvelle table sur un fichier Excel</h3>
</div>

In [62]:
#Mettre le dataset df_merge sur un fichier Excel
#Cette étape peut être utile pour partager le résultat du dataset obtenu avec les équipes.  
